# Tensoflow+MLflow Sample
 - cifar10データセットを用いて、GPUによるCNN学習・推論を試す
 - MLflowによる実験結果の記録を試す

In [57]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

GPUが利用できるか確認します。

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6705112052420506027
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 10067378176
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 9293249299753412384
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:06:00.0, compute capability: 8.6"
 xla_global_id: 416903419]

GPUが認識されているのでOK. 認識されていない場合はCUDAやcuDNNの設定を確認すること。

MLflowを有効にします。

In [3]:
import mlflow
mlflow.set_experiment("cifar10-sample")
mlflow.tensorflow.autolog()

2023/01/16 00:50:25 INFO mlflow.tracking.fluent: Experiment with name 'cifar10-sample' does not exist. Creating a new experiment.


CIFAR-10データセット―をロードします。

In [59]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

画像データを正規化します。

In [60]:
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

In [61]:
X_train.shape

(50000, 32, 32, 3)

正解ラベルをOne-Hot Encodingします。

In [62]:
num_classes = 10 # CIFAR-10は10クラス分類
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

## モデルの構築
CNNを利用した簡単なモデルを作ります。

In [8]:
input_shape = (32,32,3)

In [9]:
model = Sequential()

model.add(Conv2D(64, (3, 3), padding='same', activation="relu", input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same', activation="relu"))
model.add(Conv2D(128, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3, 3), padding='same', activation="relu"))
model.add(Conv2D(256, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# output
model.add(Flatten())
model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.25))
model.add(Dense(num_classes, activation="softmax"))

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 64)        1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 15, 15, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 15, 15, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 13, 13, 128)       147584    
                                                        

In [12]:
model.compile(
    loss="categorical_crossentropy", 
    optimizer=Adam(),
    metrics=["accuracy", "AUC"]
)

In [13]:
model.fit(X_train, y_train,
            batch_size=512,
            epochs=50,
            verbose=1,
            validation_split=0.1,
            shuffle=True
)

2023/01/16 00:51:05 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '3ede8207c71d4620ba18b3fa0e4c0c41', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/50
88/88 [==============================] - 15s 105ms/step - loss: 2.1732 - accuracy: 0.1672 - auc: 0.6398 - val_loss: 1.7900 - val_accuracy: 0.3310 - val_auc: 0.8069
Epoch 2/50
88/88 [==============================] - 6s 65ms/step - loss: 1.6515 - accuracy: 0.3925 - auc: 0.8379 - val_loss: 1.4494 - val_accuracy: 0.4734 - val_auc: 0.8798
Epoch 3/50
88/88 [==============================] - 6s 65ms/step - loss: 1.3692 - accuracy: 0.5010 - auc: 0.8925 - val_loss: 1.1852 - val_accuracy: 0.5698 - val_auc: 0.9199
Epoch 4/50
88/88 [==============================] - 6s 65ms/step - loss: 1.1712 - accuracy: 0.5796 - auc: 0.9220 - val_loss: 1.0578 - val_accuracy: 0.6224 - val_auc: 0.9378
Epoch 5/50
88/88 [==============================] - 6s 65ms/step - loss: 1.0234 - accuracy: 0.6349 - auc: 0.9406 - val_loss: 0.9022 - val_accuracy: 0.6828 - val_auc: 0.9536
Epoch 6/50
88/88 [==============================] - 6s 67ms/step - loss: 0.9110 - accuracy: 0.6775 - auc: 0.9527 - val_loss: 0.8158 -

INFO:tensorflow:Assets written to: C:\Users\Owner\AppData\Local\Temp\tmp_vu30xm7\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\Owner\AppData\Local\Temp\tmp_vu30xm7\model\data\model\assets
2023/01/16 00:56:31 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Owner\source\repos\myMLenv\.venv\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils."


In [14]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 2s 5ms/step - loss: 0.8168 - accuracy: 0.8251 - auc: 0.9636


[0.8168140053749084, 0.8251000046730042, 0.9636437892913818]

ターミナルで`samples`フォルダに移動し、`mlflow ui`を実行する。

MLflow http://127.0.0.1:5000/

アクセスすると、結果が保存されていることを確認できる。

## MLflow によるモデルの保存とロード
モデルは自動的に保存される。明示的に保存する場合は以下のようにする。

In [17]:
 #mlflow.tensorflow.log_model(model, 'cnn_4_layers')

**MLflowのバグでWindowsだとファイルパス周りがおかしくなり、MLflowのロード機能がまともに使えない。そんな・・・**

しょうがないのでtensorflowで直接ロードする

pathはmlflowで確認できる

In [58]:
model = tf.keras.models.load_model("mlruns/220654029583824991/3ede8207c71d4620ba18b3fa0e4c0c41/artifacts/model/data/model")

In [65]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 9s 4ms/step - loss: 0.8168 - accuracy: 0.8251 - auc: 0.9636


[0.8168140649795532, 0.8251000046730042, 0.9636437892913818]